In [1]:
!pip install nltk

In [5]:
import re
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
import matplotlib
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
from sklearn import metrics
import seaborn as sns

fpath="C:/Users/prane/Documents/Python and Deep Learning/LAB2/sentiment-analysis-on-movie-reviews"

import matplotlib.pyplot as plt

df_train = pd.read_csv(fpath+'/train.tsv', sep='\t')
df_test = pd.read_csv(fpath+'/test.tsv', sep='\t')
##Descriptive analyse
#this should help you to decide whether to use STOP WORDS or not.
#This part of code is just great analytical tool


##Data preprocessing
#we make text lower case and leave only letters from a-z and digits
df_train['Phrase'] = df_train['Phrase'].str.lower()
df_train['Phrase'] = df_train['Phrase'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df_test['Phrase'] = df_test['Phrase'].str.lower()
df_test['Phrase'] = df_test['Phrase'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
X_train = df_train.Phrase
y_train = df_train.Sentiment
max_fatures = 2000
tokenize = Tokenizer(num_words=max_fatures, split=' ')
tokenize.fit_on_texts(X_train.values)
X_test = df_test.Phrase
X_train = tokenize.texts_to_sequences(X_train)
X_test = tokenize.texts_to_sequences(X_test)
max_lenght = max([len(s.split()) for s in df_train['Phrase']])
X_train = pad_sequences(X_train, max_lenght)
X_test = pad_sequences(X_test, max_lenght)
print(X_train.shape)
print(X_test.shape)

##Model building
model=Sequential()
model.add(Embedding(max_fatures, output_dim=100,input_length=48))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='causal'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='causal'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.7))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(5,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model.summary()
history = model.fit(X_train, y_train, epochs=4, verbose=True,  batch_size=1024)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prane\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(156060, 48)
(66292, 48)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 48, 100)           200000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 48, 64)            32064     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 24, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 24, 64)            20544     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 12, 64)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 64)            0         
_________________________________________________________________
dense_5 (Dense)              

C:\Users\prane\Anaconda2\envs\env_full\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/4
156060/156060 [==============================] - 81s 519us/step - loss: 1.2704 - accuracy: 0.5066
Epoch 2/4
156060/156060 [==============================] - 81s 518us/step - loss: 1.1951 - accuracy: 0.5270
Epoch 3/4
156060/156060 [==============================] - 81s 519us/step - loss: 1.1200 - accuracy: 0.5648
Epoch 4/4
156060/156060 [==============================] - 82s 522us/step - loss: 1.0738 - accuracy: 0.5840
